In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir("../..")


In [3]:
import numpy as np

from numpy.typing import NDArray


from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator


In [4]:
from qml.model.gate import Gateset
from qml.model.unit import Unit, UnitManager
from qml.model.encoding import EncodingUnit, EncodingUnitManager
from qml.model.model import Model
from qml.tools.dataset import Dataset
from qml.tools.dataloader import DataLoader
from qml.tools.typing import Vector
from qml.tools.random import XRandomGenerator

from qml.trainer import evaluator as xeval


In [5]:
nq = 2
ng = 3
nx = 1
ny = 1

# dataset
train_db_size = 10
validate_db_size = 10
# loader
batch_size = 4


In [6]:
target_func = lambda x: np.sin(2 * x)
rng = XRandomGenerator()


In [7]:
xs = rng.uniform(-1, 1, train_db_size)
ys = target_func(xs)
dataset = Dataset(xs, ys)


In [15]:
uman = UnitManager(nq, ng)
model = Model(
    nq, ny,
    EncodingUnitManager.AngleEncoding(nx, nq, repeat=True),
    uman.generate_random_unit(),
    uman.generate_random_unit(),
)


In [16]:
evl = xeval.ErrorEvaluator(dataset, model, 5)
res = evl()
print(res.es)


[-1.7394551  -1.06391289 -1.19525996 -0.1184993  -0.95431398  0.22985672
 -0.82506054 -0.88903373 -0.01835265 -1.44167987]


In [17]:
evl = xeval.MSEEvaluator(dataset, model, 5)
res = evl()
print(res.loss)


1.0228533423424069


In [18]:
evl = xeval.GradientEvaluator(dataset, model, 5)
res = evl()
print(res.grads)


[[ 0.08  0.  ]
 [ 0.    0.  ]
 [ 0.    0.  ]
 [-0.1  -0.04]
 [ 0.    0.  ]
 [ 0.06 -0.04]
 [ 0.02  0.  ]
 [ 0.    0.  ]
 [ 0.08 -0.04]
 [ 0.    0.04]]


In [19]:
evl = xeval.WaveletEvaluator(xeval.Haar(), dataset, model)
res = evl()
print(res.powers)


[-0.54982582 -0.53603078  0.29449043  0.         -0.758062    0.38375348
  0.4070274   0.          0.         -0.61222699  0.15239242 -0.07384708
  0.53801112 -0.01202933  0.0616712 ]
